In [1]:
### Convert RAW DICOM data to NIFTI Image Data set
import os, sys
from os.path import join as oj


In [2]:
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from nipype.interfaces.dcm2nii import Dcm2nii

In [3]:
## This is the location of the Raw DICOM Files from BITC
StoutRawData = '/home/ehecht/BIGDATA/Stout_Homo_Faber/RAWDATA'
NiPypeOutputDir ='/EINSTEIN_BIGDATA/NIPYPE_WD/Stout_Haber/'


In [ ]:
## Figure out which directories I want to run through the preprocessing pipeline
HomoFaberImageSessions = [ x for x in os.listdir(StoutRawData) if x.startswith('HOMO-')]

print len(HomoFaberImageSessions),"Image Sets Found for HomoFaber"

In [ ]:
stoutDCM2NII_wf = pe.Workflow('stoutDCM2NII_wf') ## Initialize the workflow
stoutDCM2NII_wf.base_dir = NiPypeOutputDir  ## Tell it where to dump the results of the workflow

### Create the list of Subjects to Process
imageSession_Id_InfoSrc  = pe.Node(util.IdentityInterface(fields=['imageSessionName']),name='imageSession_Id_InfoSrc')
imageSession_Id_InfoSrc.iterables = ('imageSessionName', HomoFaberImageSessions)

In [ ]:
## Create a datasource.. this basically helps me find the individual image files and data sets for an image session
## a single image directory likely consists of DTI data, T2 images, T1 images, etc, etc
datasource = pe.Node(interface=nio.DataGrabber(infields=['imageSessionName'], outfields=['t1Mprage_dir']), name='datasource')
datasource.inputs.base_directory = StoutRawData
datasource.inputs.template = '%s/t1_mprage_sag_*'
datasource.inputs.template_args = dict( t1Mprage_dir=[['imageSessionName']])

In [ ]:
## Now create a node for the dicom converter!
dcmConvert = pe.Node(interface=Dcm2nii(),name='dcmConvert')
dcmConvert.inputs.gzip_output = False
dcmConvert.inputs.reorient = False
dcmConvert.inputs.reorient_and_crop = False

In [ ]:
stoutDCM2NII_wf.connect(imageSession_Id_InfoSrc, 'imageSessionName', datasource, 'imageSessionName')
stoutDCM2NII_wf.connect(datasource,'t1Mprage_dir', dcmConvert, 'source_dir' )



In [ ]:
converter.run()